In [2]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

# Generate json file from the keyword defined here.
the filter will filter the work or the line that you have mentioned here

In [3]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('Ransomeware.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['Ransomeware'])

KeyboardInterrupt: 

# print the txt from the file that was generated in above step

In [4]:
import json  
with open('Ransomeware.json','r') as json_file:
         for line in json_file:
             tweet = json.loads(line)
             print tweet["text"]

Ransomeware Cyber-Attacks Has Stunned The Entire World https://t.co/c7AYuqKx3G https://t.co/80tBUaEVvS
&lt;&lt;research shows a spike of ... 1,000% in Android ransomware.&gt;&gt;

https://t.co/wVW7eZegNZ https://t.co/IkmX0tn1BX
Μερική λύση αν έχουν μολυνθεί τα αρχεία σας από το #WannaCry #ransomeware Μην κάνετε επανεκκίνηση και ακολουθείστε… https://t.co/oazrlYrfZ8
Ransomeware affects Thírupathí Devasthana C https://t.co/khnBWVxAT3 TECH - COMPUTERS #affects #computers #devasthana CLASSIFIEDS ADS GRATIS
Arundhati Roy's Top Floor Machine is the latest victim of Wannacry Ransomeware 😂
Her Brain was uplinked to China, w… https://t.co/2OwipLyVwA
RT @liliputingnews: Do not reboot your PC if you get WannaCry ransomeware – try this instead https://t.co/mWstslIC1j https://t.co/A0f0XeR3Iv
North Korea Might Be Behind WannaCry 'Ransomeware' hacks | Metro.us.. Related Articles: https://t.co/RAGIVDzUmg
You Can Now Decrypt Or Patch WannaCry Ransomeware Attack On Windows, Sort Of https://t.co/02G4j5lG

# To print the token we need to define it here

In [5]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

# this will print the token which was defined in above code from the file Ramsomeware.json

In [6]:
with open('Ransomeware.json', 'r') as f:
#import io
#f=io.open('data/stream_barcelona.json', 'r', encoding='utf8' )
     for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])
        print(tokens)

[u'Ransomeware', u'Cyber-Attacks', u'Has', u'Stunned', u'The', u'Entire', u'\xa0', u'World', u'https://t.co/c7AYuqKx3G', u'https://t.co/80tBUaEVvS']
[u'&', u'lt', u';', u'&', u'lt', u';', u'research', u'shows', u'a', u'spike', u'of', u'.', u'.', u'.', u'1,000', u'%', u'in', u'Android', u'ransomware', u'.', u'&', u'gt', u';', u'&', u'gt', u';', u'https://t.co/wVW7eZegNZ', u'https://t.co/IkmX0tn1BX']
[u'\u039c', u'\u03b5', u'\u03c1', u'\u03b9', u'\u03ba', u'\u03ae', u'\u03bb', u'\u03cd', u'\u03c3', u'\u03b7', u'\u03b1', u'\u03bd', u'\u03ad', u'\u03c7', u'\u03bf', u'\u03c5', u'\u03bd', u'\u03bc', u'\u03bf', u'\u03bb', u'\u03c5', u'\u03bd', u'\u03b8', u'\u03b5', u'\u03af', u'\u03c4', u'\u03b1', u'\u03b1', u'\u03c1', u'\u03c7', u'\u03b5', u'\u03af', u'\u03b1', u'\u03c3', u'\u03b1', u'\u03c2', u'\u03b1', u'\u03c0', u'\u03cc', u'\u03c4', u'\u03bf', u'#WannaCry', u'#ransomeware', u'\u039c', u'\u03b7', u'\u03bd', u'\u03ba', u'\u03ac', u'\u03bd', u'\u03b5', u'\u03c4', u'\u03b5', u'\u03b5', u'\u0

# Print the most popular tweets in that file

In [7]:
with open('Ransomeware.json', 'r') as f:
#import io
#f=io.open('data/stream_barcelona.json', 'r', encoding='utf8' )
     for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])
        print(tokens)

[u'Ransomeware', u'Cyber-Attacks', u'Has', u'Stunned', u'The', u'Entire', u'\xa0', u'World', u'https://t.co/c7AYuqKx3G', u'https://t.co/80tBUaEVvS']
[u'&', u'lt', u';', u'&', u'lt', u';', u'research', u'shows', u'a', u'spike', u'of', u'.', u'.', u'.', u'1,000', u'%', u'in', u'Android', u'ransomware', u'.', u'&', u'gt', u';', u'&', u'gt', u';', u'https://t.co/wVW7eZegNZ', u'https://t.co/IkmX0tn1BX']
[u'\u039c', u'\u03b5', u'\u03c1', u'\u03b9', u'\u03ba', u'\u03ae', u'\u03bb', u'\u03cd', u'\u03c3', u'\u03b7', u'\u03b1', u'\u03bd', u'\u03ad', u'\u03c7', u'\u03bf', u'\u03c5', u'\u03bd', u'\u03bc', u'\u03bf', u'\u03bb', u'\u03c5', u'\u03bd', u'\u03b8', u'\u03b5', u'\u03af', u'\u03c4', u'\u03b1', u'\u03b1', u'\u03c1', u'\u03c7', u'\u03b5', u'\u03af', u'\u03b1', u'\u03c3', u'\u03b1', u'\u03c2', u'\u03b1', u'\u03c0', u'\u03cc', u'\u03c4', u'\u03bf', u'#WannaCry', u'#ransomeware', u'\u039c', u'\u03b7', u'\u03bd', u'\u03ba', u'\u03ac', u'\u03bd', u'\u03b5', u'\u03c4', u'\u03b5', u'\u03b5', u'\u0

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords") # download the stopword corpus on our computer
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'RT']

[nltk_data] Downloading package stopwords to /home/chetan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import operator 
import json
from collections import Counter
 
fname = 'Ransomeware.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        count_all.update(terms_stop)
    for word, index in count_all.most_common(5):
        print '%s : %s' % (word, index)

ा : 40
र : 36
क : 28
े : 24
म : 20


In [12]:
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]

In [13]:
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 

# collect the most popular words i the json file 
this will search in and print the most popular 10 words from the json file that we have generated. 10 different words and the repetation upto 15 times.

In [ ]:
import operator 
import json
from collections import Counter
 
fname = 'Ransomeware.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#') and term not in stop]        
        count_all.update(terms_hash)
# Print the first 10 most frequent words
print(count_all.most_common(15))

# print (plot) a bar graph from the selected words from the above code

In [ ]:
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15,10)
import matplotlib.pyplot as plt

sorted_x, sorted_y = zip(*count_all.most_common(15))
#print(sorted_x, sorted_y)

plt.bar(range(len(sorted_x)), sorted_y, width=0.75, align='center');
plt.xticks(range(len(sorted_x)), sorted_x, rotation=80);
plt.axis('tight'); 